<a href="https://colab.research.google.com/github/AbnerSilvaBarbosa/DIO-IA/blob/main/Projeto_de_Transfer_Learning_em_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importações

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import zipfile
import os
import urllib.request

# Verifica se está usando GPU
print("Using TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Using TensorFlow version: 2.15.0
Num GPUs Available:  0


In [ ]:
from google.colab import drive
import zipfile
import os

# Montar o Google Drive
drive.mount('/content/drive')

# Definir o caminho do arquivo ZIP no Google Drive
zip_file_path = '/content/drive/MyDrive/DIO IA/kagglecatsanddogs_5340.zip'
extract_path = '/content/cats_and_dogs'

# Extrair o arquivo ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Verificar se a extração foi bem-sucedida
!ls /content/cats_and_dogs

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 CDLA-Permissive-2.0.pdf   PetImages  'readme[1].txt'


In [ ]:
import os
import shutil

original_dataset_dir = '/content/cats_and_dogs/PetImages'
base_dir = '/content/cats_and_dogs_small'
os.makedirs(base_dir, exist_ok=True)

def create_small_dataset(original_dir, target_dir, category, num_samples):
    category_dir = os.path.join(original_dir, category)
    target_category_dir = os.path.join(target_dir, category)
    os.makedirs(target_category_dir, exist_ok=True)

    files = os.listdir(category_dir)[:num_samples]
    for fname in files:
        src = os.path.join(category_dir, fname)
        dst = os.path.join(target_category_dir, fname)
        shutil.copyfile(src, dst)

create_small_dataset(original_dataset_dir, base_dir, 'Cat', 1000)
create_small_dataset(original_dataset_dir, base_dir, 'Dog', 1000)

train_dir = base_dir


In [ ]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Reduzindo o tamanho da imagem
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Reduzindo o tamanho da imagem
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=64,  # Aumentando o batch size
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=64,  # Aumentando o batch size
    class_mode='binary',
    subset='validation'
)


Found 1600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [ ]:
# Treinar o modelo
callbacks = [EarlyStopping(monitor='val_loss', patience=5)]

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=callbacks
)

# Avaliar o modelo
val_loss, val_acc = model.evaluate(validation_generator)
print(f'Validation accuracy: {val_acc * 100:.2f}%')

Epoch 1/10
25/25 [==============================] - 50s 2s/step - loss: 6.8905 - accuracy: 0.8019 - val_loss: 0.4373 - val_accuracy: 0.9325
Epoch 2/10
25/25 [==============================] - 46s 2s/step - loss: 0.1842 - accuracy: 0.9494 - val_loss: 0.2503 - val_accuracy: 0.9375
Epoch 3/10
25/25 [==============================] - 50s 2s/step - loss: 0.0601 - accuracy: 0.9812 - val_loss: 0.2055 - val_accuracy: 0.9400
Epoch 4/10
25/25 [==============================] - 50s 2s/step - loss: 0.0184 - accuracy: 0.9950 - val_loss: 0.1951 - val_accuracy: 0.9450
Epoch 5/10
25/25 [==============================] - 47s 2s/step - loss: 0.0088 - accuracy: 1.0000 - val_loss: 0.1957 - val_accuracy: 0.9425
Epoch 6/10
25/25 [==============================] - 43s 2s/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 0.1967 - val_accuracy: 0.9425
Epoch 7/10
25/25 [==============================] - 43s 2s/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 0.1980 - val_accuracy: 0.9425
Epoch 8/10
25/25 [==